# Auto-Grader Step 4: Calibration, Evaluation & Demo

This notebook runs:
1. **Calibration dataset generation** - Borderline examples (scores 2-5)
2. **Calibration training** - Continue SFT from existing adapter
3. **Official evaluation** - Comprehensive metrics suite
4. **Demo** - Interactive showcase for hackathon

Run on Google Colab with GPU runtime.

## Setup

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/arabaya3/auto-grader.git
%cd auto-grader

In [ ]:
# Install dependencies
!pip install -q transformers>=4.36.0 accelerate>=0.25.0 peft>=0.7.0 bitsandbytes>=0.41.0 trl>=0.7.0 datasets scipy

In [ ]:
# Verify installations
import torch
import transformers
import trl
import peft

print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"TRL: {trl.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

## Step 4.1: Generate Calibration Dataset

Creates borderline examples (scores 2-5) to improve score accuracy.

In [ ]:
from src.data.build_calibration_dataset import (
    build_calibration_dataset,
    build_adversarial_dataset,
    quality_check
)

# Generate calibration data (100 examples)
build_calibration_dataset(
    output_file="data/calibration.jsonl",
    num_examples=100,
    seed=42
)

# Generate adversarial test data
build_adversarial_dataset(
    output_file="data/adversarial.jsonl"
)

# Quality check
quality_check("data/calibration.jsonl")

In [ ]:
# Preview calibration data
import json

with open("data/calibration.jsonl") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        example = json.loads(line)
        print(f"\n=== Example {i+1} ===")
        print(f"Score: {example['label']['score']}")
        print(f"Prompt: {example['prompt'][:80]}...")
        print(f"Reasoning: {example['label']['reasoning'][:100]}...")

## Step 4.2: Calibration Training

Continue SFT from Step 3 adapter with calibration data.

In [ ]:
# Check if we have the Step 3 adapter
import os

adapter_path = "outputs/judge_sft_lora/final_adapter"

if not os.path.exists(adapter_path):
    print("Step 3 adapter not found!")
    print("Please run notebook 02_train_sft_qlora.ipynb first.")
else:
    print(f"Step 3 adapter found at: {adapter_path}")
    print("Ready for calibration training!")

In [ ]:
# Run calibration training (if Step 3 adapter exists)
# This continues training from the existing adapter

from src.training.sft_calibration_train import CalibrationConfig, calibration_train

config = CalibrationConfig(
    base_model="Qwen/Qwen2.5-1.5B-Instruct",
    adapter_path="outputs/judge_sft_lora/final_adapter",
    output_dir="outputs/judge_sft_lora_calibrated",
    num_train_epochs=1,
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    use_4bit=True,
)

# Only run if adapter exists
if os.path.exists(adapter_path):
    calibrated_adapter = calibration_train(config, "data/calibration.jsonl")
    print(f"\nCalibrated adapter saved to: {calibrated_adapter}")
else:
    print("Skipping calibration - Step 3 adapter needed first")

## Step 4.3: Official Evaluation

Run comprehensive evaluation with metrics.

In [ ]:
from src.eval.eval_official import (
    EvalConfig, load_model, evaluate_gold_set, 
    evaluate_adversarial, generate_report
)

# Select which adapter to evaluate
# Options: Step 3 or Calibrated (Step 4)
ADAPTER_TO_EVAL = "outputs/judge_sft_lora/final_adapter"  # Step 3
# ADAPTER_TO_EVAL = "outputs/judge_sft_lora_calibrated/final_adapter"  # Step 4

# Check which exists
if os.path.exists("outputs/judge_sft_lora_calibrated/final_adapter"):
    print("Using calibrated adapter (Step 4)")
    ADAPTER_TO_EVAL = "outputs/judge_sft_lora_calibrated/final_adapter"
elif os.path.exists("outputs/judge_sft_lora/final_adapter"):
    print("Using Step 3 adapter")
    ADAPTER_TO_EVAL = "outputs/judge_sft_lora/final_adapter"
else:
    print("No adapter found - will evaluate baseline model")
    ADAPTER_TO_EVAL = None

print(f"\nAdapter: {ADAPTER_TO_EVAL}")

In [ ]:
# Run official evaluation
config = EvalConfig(
    base_model="Qwen/Qwen2.5-1.5B-Instruct",
    adapter_path=ADAPTER_TO_EVAL,
    use_4bit=True,
)

# Load model
model, tokenizer = load_model(config)

# Evaluate gold set
metrics, results = evaluate_gold_set(
    model, tokenizer, "data/gold_tests.jsonl", config
)

# Print summary
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"JSON Validity: {metrics.json_validity_rate:.1f}%")
print(f"Score Accuracy: {metrics.score_accuracy:.1f}%")
print(f"Score MAE: {metrics.score_mae:.2f}")
print(f"Score Within ±1: {metrics.score_within_1_rate:.1f}%")
if metrics.pearson_r:
    print(f"Pearson r: {metrics.pearson_r:.3f}")

In [ ]:
# Evaluate adversarial (robustness)
if os.path.exists("data/adversarial.jsonl"):
    metrics.adversarial_results = evaluate_adversarial(
        model, tokenizer, "data/adversarial.jsonl", config
    )

# Generate reports
json_path, md_path = generate_report(
    metrics, results, "outputs/eval_results", config
)

print(f"\nReports saved:")
print(f"  {json_path}")
print(f"  {md_path}")

In [ ]:
# Display markdown report
from IPython.display import Markdown, display

with open(md_path) as f:
    report_content = f.read()

display(Markdown(report_content))

## Step 4.4: Demo for Hackathon

Interactive demo showcasing the judge model.

In [ ]:
from src.demo.demo_run import JudgeDemo, DEMO_CASES

# Create demo with best available adapter
demo = JudgeDemo(
    base_model="Qwen/Qwen2.5-1.5B-Instruct",
    adapter_path=ADAPTER_TO_EVAL,
    use_4bit=True,
    with_confidence=True
)

# Load model
demo.load_model()

In [ ]:
# Run all demo cases
results = demo.run_all_demos()

# Print summary table
demo.print_summary_table(results)

In [ ]:
# Interactive demo - try your own examples!
# Uncomment to run:
# demo.interactive_mode()

## Step 4.5: Enhanced Features Demo

Showcase winning features: confidence, explainability, strict JSON.

In [ ]:
from src.inference_enhanced import EnhancedJudge

# Create enhanced judge
enhanced = EnhancedJudge(
    base_model="Qwen/Qwen2.5-1.5B-Instruct",
    adapter_path=ADAPTER_TO_EVAL,
    use_4bit=True,
    max_retries=3  # For strict JSON enforcement
)

enhanced.load_model()

In [ ]:
# Demo: Confidence Scoring
print("\n" + "="*50)
print("CONFIDENCE SCORING DEMO")
print("="*50)

test_cases = [
    {
        "prompt": "What is the capital of France?",
        "response": "The capital of France is Paris."
    },
    {
        "prompt": "What is quantum computing?",
        "response": "Quantum computing uses quantum bits or qubits..."
    },
]

for case in test_cases:
    result = enhanced.judge_with_confidence(case["prompt"], case["response"])
    print(f"\nPrompt: {case['prompt']}")
    print(f"Score: {result.score}/5 | Confidence: {result.confidence:.2f}")
    print(f"Reasoning: {result.reasoning[:100] if result.reasoning else 'N/A'}...")

In [ ]:
# Demo: Explainability Mode
print("\n" + "="*50)
print("EXPLAINABILITY DEMO")
print("="*50)

result = enhanced.judge_with_explanation(
    "Explain photosynthesis",
    "Photosynthesis is when plants make food from sunlight. They take in CO2 and water, and produce glucose and oxygen."
)

print(f"\nScore: {result.score}/5")
print(f"Confidence: {result.confidence:.2f}")
print(f"\nExplanation:")
if result.explanation:
    print(f"  Summary: {result.explanation['summary']}")
    print(f"  Score Meaning: {result.explanation['score_meaning']}")
    print(f"  Criteria Analyzed: {len(result.explanation['criteria_analysis'])}")

In [ ]:
# Demo: Strict JSON Enforcement
print("\n" + "="*50)
print("STRICT JSON ENFORCEMENT DEMO")
print("="*50)

result = enhanced.judge_strict(
    "Write a haiku about coding",
    "Bugs in the code / Debugging all night long / Fixed at dawn's light"
)

print(f"\nJSON Valid: {result.json_valid}")
print(f"Retry Count: {result.retry_count}")
print(f"Score: {result.score}")

## Summary & Next Steps

**Completed:**
- ✅ Calibration dataset generation
- ✅ Calibration training (if Step 3 adapter available)
- ✅ Official evaluation with comprehensive metrics
- ✅ Demo runner for hackathon presentation
- ✅ Enhanced features (confidence, explainability, strict JSON)

**Artifacts:**
- `outputs/eval_results/judge_final_report.json` - Structured metrics
- `outputs/eval_results/judge_final_report.md` - Markdown summary
- `outputs/judge_sft_lora_calibrated/` - Calibrated adapter (if training ran)

**For Hackathon:**
1. Show JSON validity improvement (100%)
2. Demonstrate confidence scores
3. Run interactive demo
4. Show robustness against adversarial inputs

In [ ]:
print("\n" + "="*60)
print("AUTO-GRADER STEP 4 COMPLETE!")
print("="*60)
print("\nReady for hackathon demo. Key achievements:")
print("  • JSON validity: 100%")
print("  • Confidence scoring")
print("  • Explainability mode")
print("  • Strict JSON enforcement")
print("  • Robustness testing")